In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, scale

In [2]:
df = pd.read_csv('./baseline_data.csv',header=0,low_memory=False)

In [3]:
Catdata = df.loc[0:,'PTETHCAT':'APOE4'].join(df['PTGENDER']).join(df['imputed_genotype'])
for i in Catdata.columns:
    the_value = str(Catdata[i].mode().values[0]) # find the most common value in the column
    Catdata[i].replace('NaN',the_value,inplace = True)
    dummy_data = pd.get_dummies(Catdata[i], prefix=i+"_", drop_first=True) # https://www.jianshu.com/p/c324f4101785
    Catdata = pd.concat([Catdata, dummy_data], axis=1)
    Catdata.drop(i, axis=1, inplace=True)

Numdata = [];
feature_dis = 25
for i in range(feature_dis):
    Numdata.append(df.iloc[:,i:2150:feature_dis].join(df['AGE']).join(df['PTEDUCAT']).join(df['MMSE']))
#for i in Numdata[10].columns:
    #print(i)
labels = df['DX.bl']
le = LabelEncoder()
le.fit(labels) # http://sofasofa.io/forum_main_post.php?postid=1001659
labels = le.transform(labels)
labels = labels.astype('float') # 1 CN, 0 AD, 2 LMCI


##########################################
data = Numdata[0].join(Catdata)
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 1)
import math
acc_train, acc_test, params = [], [], []
for c in np.arange(1, 100):
    print(c)
    logisticRegr1 = LogisticRegression(C=c*0.01, random_state=0)
    logisticRegr1.fit(X_train, y_train)
    acc_train.append(logisticRegr1.score(X_train,y_train))
    acc_test.append(logisticRegr1.score(X_test,y_test))
    params.append(c*0.01)

import matplotlib.pyplot as plt

plt.plot(params, acc_train, color='blue', marker='x', label='training set')
plt.plot(params, acc_test, color='green',  marker='o', label='test set')
plt.ylabel('Accuray')
plt.xlabel('C')
plt.legend(loc='right')
#plt.xscale('log')
plt.show() 
########################################################

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, scale
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
logisticRegr=[]
for i in range(feature_dis):
    data = Numdata[i].join(Catdata)
    scaler.fit(data)
    data = pd.DataFrame(scaler.transform(data))
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 2)
    logisticRegr.append(LogisticRegression())
    logisticRegr[i].fit(X_train,y_train)
    # accuracy on train model
    #print(logisticRegr[i].score(X_train,y_train))
    # accuracy on test model
    #print(logisticRegr[i].score(X_test,y_test))

In [5]:
y_predict=[]
for i in range(feature_dis):
    y_predict.append(logisticRegr[i].predict(X_test))
YY_predict_all=pd.DataFrame(y_predict)
YY_predict=[]
for i in YY_predict_all.columns:
    the_value = YY_predict_all[i].mode().values[0] # find the most common value in the column
    YY_predict.append(the_value)
    

In [6]:
# accuracy on test model
count=0
for i in range(len(YY_predict)):
    if YY_predict[i]==y_test[i]:
        count+=1
accuarcy = count/len(YY_predict)
print(accuarcy)

0.6904761904761905
